In [21]:
import pandas as pd
import sklearn
import torch
import numpy as np
from  sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import Dataset
from torch import optim

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def preprocess_data(data_path, with_dummies=True):
    train_data = pd.read_csv(data_path)
    train_data.drop(columns=["id"], inplace=True)
    train_data[["święto", "dzień_roboczy"]] = train_data[["święto", "dzień_roboczy"]].astype(int)
    train_data[['year', 'month', 'day']] = train_data['data'].str.split('-', expand=True).astype(int)
    train_data.drop("data", axis=1, inplace=True)
    if with_dummies:
        return pd.get_dummies(train_data, columns=['pogoda'], dtype=int)
    return train_data

train_data_with_dummies = preprocess_data("bit-x-adata/train.csv")

X = train_data_with_dummies.drop("studenty_ms", axis=1).values
Y = train_data_with_dummies["studenty_ms"].values.reshape(-1,1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

final_test_data = preprocess_data("bit-x-adata/test.csv")


In [23]:
train_data_with_dummies.head()

,święto,dzień_roboczy,temperatura,odczuwalna_temperatura,wilgotność,prędkość_wiatru,studenty_ms,year,month,day,pogoda_lekkie_opady,pogoda_pochmurno,pogoda_ładna_pogoda
0,0,1,2,0,39,23,12,2020,1,2,0,0,1
1,0,1,-2,0,47,7,15,2020,1,3,0,0,1
2,0,0,-1,-3,47,11,6,2020,1,5,0,0,1
3,1,0,-2,-4,51,11,18,2020,1,6,0,0,1
4,0,1,-6,-7,36,16,11,2020,1,8,0,0,1


In [24]:
X_train

array([[-0.20739034,  0.66269838,  1.15425927, ..., -0.3352392 ,
        -0.50321961,  0.65947801],
       [-0.20739034,  0.66269838,  0.49409919, ...,  2.98294472,
        -0.50321961, -1.51635079],
       [-0.20739034,  0.66269838,  1.26428595, ..., -0.3352392 ,
        -0.50321961,  0.65947801],
       ...,
       [-0.20739034,  0.66269838,  1.92444603, ..., -0.3352392 ,
        -0.50321961,  0.65947801],
       [-0.20739034, -1.50898212,  0.38407251, ..., -0.3352392 ,
         1.98720396, -1.51635079],
       [-0.20739034,  0.66269838,  0.38407251, ..., -0.3352392 ,
         1.98720396, -1.51635079]], shape=(310, 12))

In [25]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()

In [26]:
X_train.shape[1]

12

In [27]:
# class GepardPred(nn.Module):
#     def __init__(self, input_size: int, dropout_rate: float = 0.5):
#         super().__init__()

#         self.model = nn.Sequential(
#             nn.Linear(input_size, 256),
#             nn.ReLU(),
#             nn.Dropout(dropout_rate),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Dropout(dropout_rate),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Dropout(dropout_rate),
#             nn.Linear(256, 1),

#         )

#     def forward(self, x):
#         return self.model(x)

#     def predict_int(self, x, min_value: int = 0):
#         """
#         Run inference and return non-negative integer predictions.
#         x may be a numpy array or a torch.Tensor.
#         """
#         was_training = self.training
#         self.eval()
#         with torch.no_grad():
#             if not isinstance(x, torch.Tensor):
#                 inputs = torch.tensor(x, dtype=torch.float32)
#             else:
#                 inputs = x.float()
#             out = self.model(inputs)

#             # ensure non-negative, round to nearest integer and cast to long
#             out = torch.relu(out)
#             ints = torch.round(out).to(torch.long).squeeze(-1)

#             # enforce min_value if needed
#             if min_value != 0:
#                 ints = torch.max(ints, torch.tensor(min_value, dtype=ints.dtype))

#         if was_training:
#             self.train()

#         return ints

In [28]:
# # curr best
# # model loss = 1.0727
# class GepardPred(nn.Module):
#     def __init__(self, input_size: int, dropout_rate: float = 0.5):
#         super().__init__()

#         self.model = nn.Sequential(
#             nn.Linear(input_size, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Linear(256, 1),
#             nn.ReLU(),
#         )

#     def forward(self, x):
#         return self.model(x)

In [29]:
# model loss = 1.0717
class GepardPred(nn.Module):
    def __init__(self, input_size: int, dropout_rate: float = 0.5):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.model(x)

In [30]:
# model loss = ...
class GepardPred(nn.Module):
    def __init__(self, input_size: int, dropout_rate: float = 0.5):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 1),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.model(x)

In [31]:
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

In [32]:
class MyDataset(Dataset):
    def __init__(self, data, y):
        super().__init__()

        self.data = data
        self.y = y

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.y[idx]



In [33]:
from torch.utils.data import DataLoader

learning_rate = 1e-3
dropout_p = 0.5
l2_reg = 1e-4
batch_size = 128
max_epochs = 300

early_stopping_patience = 4

In [34]:
from copy import deepcopy

In [35]:
def evaluate_model(model: GepardPred, X, y, loss_fn):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X, dtype=torch.float32)
        targets = torch.tensor(y, dtype=torch.float32)

        # outputs = model.predict_int(inputs)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

    return {"loss": loss.item()}

In [36]:
dropout_p = 0.3
learning_rate = 1e-3

model = GepardPred(
    input_size=X_train.shape[1],
    dropout_rate=dropout_p
)

optimizer = optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=l2_reg
)

train_dataset = MyDataset(
    X_train,
    y_train
)

data_loader = DataLoader(
    train_dataset,
    batch_size
)

loss_fn = RMSLELoss()

best_model = model
best_threshold = None
patience = 10
steps_without_improvement = 0
best_loss = np.inf

for epoch in range(max_epochs):
    model.train()

    for X_batch, y_batch in data_loader:
        y_pred = model(X_batch)

        loss = loss_fn(y_pred, y_batch)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    valid_metrics = evaluate_model(model, X_valid, y_valid, loss_fn)

    valid_loss = valid_metrics["loss"]

    print(f"Epoch {epoch} train loss: {loss.item():.4f}, eval loss {valid_loss:.4f}")

    if valid_loss < best_loss:
        best_loss = valid_loss
        best_model = deepcopy(model)

        steps_without_improvement = 0
    else:
        steps_without_improvement += 1
        if steps_without_improvement >= patience:

            print(f"Early stopping at epoch {epoch}")
            break


Epoch 0 train loss: 2.9940, eval loss 2.8540
Epoch 1 train loss: 2.5256, eval loss 2.3828
Epoch 2 train loss: 2.0786, eval loss 1.9418
Epoch 3 train loss: 1.6645, eval loss 1.6408
Epoch 4 train loss: 1.4065, eval loss 1.5004
Epoch 5 train loss: 1.3159, eval loss 1.4732
Epoch 6 train loss: 1.3401, eval loss 1.4941
Epoch 7 train loss: 1.3513, eval loss 1.5161
Epoch 8 train loss: 1.3699, eval loss 1.5176
Epoch 9 train loss: 1.3455, eval loss 1.4929
Epoch 10 train loss: 1.3393, eval loss 1.4442
Epoch 11 train loss: 1.2731, eval loss 1.3787
Epoch 12 train loss: 1.2330, eval loss 1.3037
Epoch 13 train loss: 1.1458, eval loss 1.2280
Epoch 14 train loss: 1.1077, eval loss 1.1608
Epoch 15 train loss: 1.0737, eval loss 1.1081
Epoch 16 train loss: 1.0715, eval loss 1.0724
Epoch 17 train loss: 1.0409, eval loss 1.0508
Epoch 18 train loss: 0.9795, eval loss 1.0391
Epoch 19 train loss: 1.0004, eval loss 1.0365
Epoch 20 train loss: 1.0229, eval loss 1.0438
Epoch 21 train loss: 1.0105, eval loss 1.054